# V17 (gated 82+83): Ephemeris reliability

Re-runs the phase-shift null and nearby-period neighborhood checks on the gated (82+83) dataset.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

sectors_used = [82, 83]
ds = sh.load_dataset()
lc = sh.stitch_pdcsap_sectors(ds, sectors_used)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V17')

print(json.dumps({
  'sectors_used': sectors_used,
  'depth_ppm': depth_ppm,
  'depth_err_ppm': depth_err_ppm,
  'status': r.status,
  'flags': list(r.flags),
  'metrics': r.metrics,
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_err_ppm": 11.153641839855947,
  "depth_ppm": 264.57118442002604,
  "flags": [],
  "metrics": {
    "depth_hat_ppm": 267.1192365173071,
    "depth_sigma_ppm": 9.483800429830776,
    "effective_n_points": 385.2553844525222,
    "max_ablation_score_drop_fraction": 0.030711572964288707,
    "n_in_transit": 487,
    "null_percentile": 0.9950248756218906,
    "period_neighborhood_best_period_days": 14.245220874480001,
    "period_neighborhood_best_score": 28.22545985045269,
    "period_neighborhood_second_best_score": 28.16584327071014,
    "period_peak_to_next_ratio": 1.0021166268366104,
    "phase_shift_null_p_value": 0.004975124378109453,
    "phase_shift_null_z": 5.3034546716866755,
    "score": 28.16584327071014,
    "top_5_fraction_abs": 0.0
  },
  "sectors_used": [
    82,
    83
  ],
  "status": "ok"
}
```

</details>


In [ ]:
import json
import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_ephemeris_reliability

step_id = '26_gated_82_83_v17_ephemeris_reliability'
fname = 'V17_ephemeris_reliability.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(9, 5))
    plot_ephemeris_reliability(r, ax=ax)
    ax.set_title('V17 (gated 82+83): Ephemeris reliability')
    fig.tight_layout()
    fig.savefig(run_path, dpi=160, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=160, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/26_gated_82_83_v17_ephemeris_reliability/V17_ephemeris_reliability.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/26_gated_82_83_v17_ephemeris_reliability/V17_ephemeris_reliability.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/26_gated_82_83_v17_ephemeris_reliability/V17_ephemeris_reliability.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** phase-shift null p-value remains low (~0.005), but the period neighborhood peak-to-next ratio is ~1.00 (very modest).
- **Why this is useful:** distinguishes “phase-coherent” from “uniquely tied to this exact period”.
- **Interpretation:** the signal remains phase-coherent, but period uniqueness is weak in the gated dataset.
- **Next step:** V18 (gated) sensitivity sweep to quantify how strongly preprocessing choices move the score/depth.

</details>
